# Junior ML contest

In [1]:
import numpy as np
import pandas as pd

# import seaborn as sns
# import matplotlib.pyplot as plt

from sklearn.tree import  DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

import torch

comp_students = pd.read_csv("comp_students.csv")
comp_portrait = pd.read_csv("comp_portrait.csv")
comp_marks = pd.read_csv("comp_marks.csv")
comp_disc = pd.read_csv("comp_disc.csv")
train = pd.read_csv("train (1).csv")
comp_teachers = pd.read_csv("comp_teachers.csv")
test = pd.read_csv("test (1).csv")
sample_submission = pd.read_csv("sample_submission (1).csv")

## EDA

## Make data for model training

In [2]:
semester_of_disc = comp_disc[["DISC_ID", "SEMESTER"]]
semester_of_disc = semester_of_disc.groupby("DISC_ID").min()
semester_of_disc

,SEMESTER
DISC_ID,
5431184050471578,2.0
6947608674753041,3.0
10928733136090636,6.0
13187948132681189,2.0
18938951954663142,2.0
...,...
18423222245760976082,2.0
18423613269743954405,3.0
18431371490397109397,6.0


In [3]:
def erase_disc(row, piv_tab_cols):
    # TODO проверить правильно ли оно работает
    """
    Erase the mark for the same subject from the line with the subject
    :param row: row of students_table
    :param piv_tab_cols: name of columns pivot_table
    :return:
    """
    disc = row["DISC_ID"]
    if disc not in piv_tab_cols:
        return row
    row[disc] = 0
    return row

In [4]:
def erase_future_disc(row, piv_tab_cols, semesters):
    cur_sem = row["SEMESTER"]
    disc_to_erase = piv_tab_cols[semesters >= cur_sem]
    row[disc_to_erase] = 0
    return row

In [5]:
%%time
# students_table = pd.merge(comp_portrait, comp_marks, how="outer")
students_table = pd.merge(train, comp_portrait, how="inner", on="ISU")
students_table = pd.merge(students_table, comp_marks, how="left", on=["ISU", "DISC_ID", "SEMESTER", "ST_YEAR", "TYPE_NAME"])
# deal with nan
for col in ["ADMITTED_EXAM_1", "ADMITTED_EXAM_2", "ADMITTED_EXAM_3"]:
    students_table[col].fillna(students_table[col].mean(), inplace=True)
students_table[["MARK", "PRED_ID", "MAIN_PLAN"]] = students_table[["MARK", "PRED_ID", "MAIN_PLAN"]].fillna(0)
# print(students_table.info())
# adding information about other disciplines
count_disc = students_table.groupby(by="DISC_ID", ).count()["ISU"].sort_values(ascending=False)
dics_to_add_to_students_table = count_disc[count_disc > 300].index
students_table["MARK"].replace(["незач", "неявка"], "2", inplace=True)
students_table["MARK"].replace(["зачет", "осв"], "4", inplace=True)
students_table["MARK"] = students_table["MARK"].astype("int")
p_t = pd.pivot_table(students_table, values="MARK", index="ISU", columns="DISC_ID", aggfunc=np.mean)
p_t = p_t[dics_to_add_to_students_table]
p_t.fillna(0, inplace=True)
students_table = pd.merge(students_table, p_t, how="outer", on="ISU")

# Erase mark of DISC_ID
# students_table = students_table.apply(erase_disc, axis=1, args=[p_t.columns])

# Erase marks of future disciplines, regarding current DICS_ID
semesters_of_each_disc = semester_of_disc[semester_of_disc.index.isin(p_t.columns)]["SEMESTER"]
print(semesters_of_each_disc.shape)
students_table = students_table.apply(erase_future_disc, axis=1, args=[p_t.columns, semesters_of_each_disc])

# make category
categorical_columns = ["CITIZENSHIP", "GENDER", "EXAM_TYPE", "EXAM_SUBJECT_1", "EXAM_SUBJECT_2", "EXAM_SUBJECT_3",
                       "REGION_ID", "ST_YEAR", "SEMESTER", "TYPE_NAME", "ADMITTED_SUBJECT_PRIZE_LEVEL"]
for col in categorical_columns:
    students_table[col] = students_table[col].astype('category')
dummies = pd.get_dummies(students_table[categorical_columns], prefix=categorical_columns)
students_table.drop(columns=categorical_columns, inplace=True)
students_table = pd.concat([students_table, dummies], axis="columns")
# rename columns
students_table.columns = [str(x_i) for x_i in students_table.columns]
print("******")
students_table

(138,)
******
CPU times: total: 2min 36s
Wall time: 2min 37s


,ISU,DISC_ID,DEBT,ADMITTED_EXAM_1,ADMITTED_EXAM_2,ADMITTED_EXAM_3,MARK,MAIN_PLAN,PRED_ID,9028324525774925341,...,TYPE_NAME_Дифференцированный зачет,TYPE_NAME_Зачет,TYPE_NAME_Курсовой проект,TYPE_NAME_Экзамен,ADMITTED_SUBJECT_PRIZE_LEVEL_0.0,ADMITTED_SUBJECT_PRIZE_LEVEL_1.0,ADMITTED_SUBJECT_PRIZE_LEVEL_2.0,ADMITTED_SUBJECT_PRIZE_LEVEL_3.0,ADMITTED_SUBJECT_PRIZE_LEVEL_4.0,ADMITTED_SUBJECT_PRIZE_LEVEL_ЕГЭ
0,17623928044460629977,10502311854018326223,0,78.0,79.0,91.0,4,4.664713e+18,5.087927e+18,0.0,...,0,1,0,0,0,0,0,0,0,1
1,17623928044460629977,1601392918367593206,0,78.0,79.0,91.0,4,4.664713e+18,5.087927e+18,0.0,...,0,1,0,0,0,0,0,0,0,1
2,17623928044460629977,9559803959325174929,0,78.0,79.0,91.0,4,4.664713e+18,5.087927e+18,0.0,...,0,1,0,0,0,0,0,0,0,1
3,17623928044460629977,8955667882044263414,0,78.0,79.0,91.0,4,4.664713e+18,5.087927e+18,0.0,...,0,1,0,0,0,0,0,0,0,1
4,17623928044460629977,17741967398854095262,0,78.0,79.0,91.0,3,4.664713e+18,5.087927e+18,0.0,...,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186439,4183409784835048954,11780410418462562766,0,70.0,64.0,91.0,0,0.000000e+00,0.000000e+00,0.0,...,0,1,0,0,0,0,0,0,0,1
186440,4183409784835048954,5390045268612005024,0,70.0,64.0,91.0,0,0.000000e+00,0.000000e+00,0.0,...,0,1,0,0,0,0,0,0,0,1
186441,4183409784835048954,13380346061330718296,0,70.0,64.0,91.0,0,0.000000e+00,0.000000e+00,0.0,...,0,1,0,0,0,0,0,0,0,1
186442,4183409784835048954,9075687484357010791,0,70.0,64.0,91.0,0,0.000000e+00,0.000000e+00,0.0,...,0,1,0,0,0,0,0,0,0,1


In [6]:
p_t.shape

(5095, 138)

## Check completeness of data

In [7]:
%%time
errors = []
for idx, row_ in students_table.iterrows():
    disc = row_["DISC_ID"]
    if str(disc) in students_table.columns:
        if students_table[str(disc)] != 0:
           errors.append([idx, disc])
print(errors)

[]
CPU times: total: 10.3 s
Wall time: 10.3 s


In [8]:
print(test["ISU"].isin(train["ISU"]).value_counts().all())
print(test["ISU"].isin(students_table["ISU"]).value_counts().all())
print(students_table["ISU"].isin(train["ISU"]).value_counts().all())

True
True
True


## Making train/test data

In [9]:
data_x = students_table.drop(columns=["MAIN_PLAN", "PRED_ID", "ISU", "MARK"])
data_y = data_x["DEBT"]
data_x = data_x.drop(columns=["DEBT"])

indexes = np.arange(0, data_x.shape[0])
np.random.shuffle(indexes)
train_x = data_x.iloc[indexes, :]
train_y = data_y.iloc[indexes]

split_indexes = (train_x["ST_YEAR_2020"] == 1) | (train_x["ST_YEAR_2019"] == 1)
# split_indexes = (train_x["ST_YEAR_2018"] == 1) | (train_x["ST_YEAR_2019"] == 1)
# split_indexes = train_x["ST_YEAR_2020"] == 1
test_x = train_x[~split_indexes]
test_y = train_y[~split_indexes]
train_x = train_x[split_indexes]
train_y = train_y[split_indexes]
print(train_x.shape, train_y.shape)

(154029, 282) (154029,)


In [10]:
# count of 0 class and 1 class
(train_y.shape[0] - train_y.sum()), train_y.sum()

(147374, 6655)

## Baseline on Decision Tree

In [11]:
baseline_decision_tree = DecisionTreeClassifier()
baseline_decision_tree.fit(train_x, train_y)

pred_y = baseline_decision_tree.predict(train_x)
train_metric = f1_score(train_y, pred_y)
print("train baseline metric", train_metric)

pred_y = baseline_decision_tree.predict(test_x)
test_metric = f1_score(test_y, pred_y)
print("test baseline metric", test_metric)
print(baseline_decision_tree.get_n_leaves(), baseline_decision_tree.get_depth())

train baseline metric 0.9868330923205724
test baseline metric 0.272
5315 62


## Train

In [12]:
device = torch.device("cuda")
torch.set_printoptions(sci_mode=False)

In [13]:
import torchmetrics
from torch.utils.data import DataLoader, TensorDataset


class Network(torch.nn.Module):
    def __init__(self, inp_shape):
        super().__init__()
        self.linear1 = torch.nn.Linear(inp_shape, 256)
        self.linear2 = torch.nn.Linear(256, 256)
        self.linear3 = torch.nn.Linear(256, 2)
        self.ReLU = torch.nn.ReLU()
        self.Sigmoid = torch.nn.Sigmoid()
        self.SoftMax = torch.nn.Softmax(dim=-1)

    def forward(self, x):
        out = self.ReLU(self.linear1(x))
        out = self.ReLU(self.linear2(out))
        out = self.SoftMax(self.linear3(out))
        return out

    def predict(self, x):
        return self(x)


def fit(model, epochs, loss_func, opt, train_dl, test_dl):
    f1_func = torchmetrics.F1(num_classes=2, average="macro").to(device)
    precision_recall = torchmetrics.functional.precision_recall
    x_test, y_test = test_dl.dataset.tensors[0], test_dl.dataset.tensors[1]
    for epoch in range(epochs):
        for x, y in train_dl:
            y_predict = model(x)
            loss = loss_func(y_predict, y)
            opt.zero_grad()
            loss.backward()
            opt.step()
        with torch.no_grad():
            if epoch % 2 == 0:
                y_predict = model(x_test)
                f1 = round(f1_func(y_predict, y_test).item(), 3)
                print(f"train loss {round(loss.item(), 5)} test f1 {f1}")
                # print(f"train loss {round(loss.item(), 5)}")
    with torch.no_grad():
        y_predict = model(x_test)
        f1 = f1_func(y_predict, y_test)
        p_r = precision_recall(y_predict, y_test, average="macro", num_classes=2)
        print(f"TEST RESULT:\n\tResult f1={round(f1.item(), 3)}\n\tprecision and recall={round(p_r[0].item(), 3), round(p_r[1].item(), 3)}")
    return model


In [14]:
U, S, V = torch.pca_lowrank(torch.tensor(train_x.to_numpy(), device=device))
U_cut = torch.reshape(U[:, 0], (U.shape[0], 1))
V_cut = torch.reshape(V[:, 0], (V.shape[0], 1))
print(U_cut.shape, S.shape, V_cut.shape)

torch.Size([154029, 1]) torch.Size([6]) torch.Size([282, 1])


In [15]:
f1_func = torchmetrics.F1(num_classes=2, average="macro").to(device)


# deal with imbalance of data
koef_of_imbalance = (train_y.shape[0] - train_y.sum()) // train_y.sum()
minority_class = train_x.loc[train_y == 1, :]
minority_class = pd.concat([minority_class] * koef_of_imbalance, axis=0)
print(minority_class.shape, train_x.shape)
balanced_train_x = pd.concat([train_x, minority_class], axis=0)
balanced_train_y = pd.concat([train_y, pd.Series([1] * minority_class.shape[0])], axis=0)

indexes = np.arange(0, balanced_train_x.shape[0])
np.random.shuffle(indexes)
balanced_train_x = balanced_train_x.iloc[indexes, :]
balanced_train_y = balanced_train_y.iloc[indexes]

# normalization
normal_train_x = balanced_train_x.to_numpy()
print(np.mean(normal_train_x, axis=0).shape)
normal_train_x = (normal_train_x - np.mean(normal_train_x, axis=0)) / np.std(normal_train_x, axis=0)

print(balanced_train_x.shape, balanced_train_y.shape)


(146410, 282) (154029, 282)
(282,)
(300439, 282) (300439,)


C:\Users\Norma\AppData\Local\Temp\ipykernel_9380\2757370389.py:20: RuntimeWarning: invalid value encountered in true_divide
  normal_train_x = (normal_train_x - np.mean(normal_train_x, axis=0)) / np.std(normal_train_x, axis=0)


In [16]:
balanced_train_y[balanced_train_y == 1].shape[0], balanced_train_y[balanced_train_y == 0].shape[0]

(153065, 147374)

In [17]:
train_dataset = TensorDataset(torch.tensor(normal_train_x, device=device), torch.tensor(balanced_train_y.to_numpy(), device=device))
# train_dataset = TensorDataset(torch.tensor(normal_train_x, device=device), torch.tensor(train_y.to_numpy(), device=device))
test_dataset = TensorDataset(torch.tensor(test_x.to_numpy(), device=device), torch.tensor(test_y.to_numpy(), device=device))
train_data_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_data_loader = DataLoader(test_dataset, batch_size=test_x.shape[0], shuffle=True)

In [18]:
model = Network(train_x.shape[1]).to(device=device, dtype=torch.double)
# loss_function = torch.nn.CrossEntropyLoss(weight=torch.tensor([0.1, 0.9], device=device, dtype=torch.double))
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.1, weight_decay=0.01)

epochs = 10
# TODO ПРОВЕРИТЬ ДАТАСЕТ НА ЁБНУТОСТЬ. ПОЧЕМУ ВСЁ ВРЕМЯ ОДНО ЗНАЧЕНИЕ, НО ОТ 0, ТО 1?
model = fit(model, epochs, loss_function, optimizer, train_data_loader, test_data_loader)

# pred_y = model(test_data_loader.dataset.tensors[0])
# test_metric = f1_func(pred_y, torch.tensor(test_data_loader.dataset.tensors[1], dtype=torch.int32))
# print("test_metric", test_metric)

pred_y = model(train_data_loader.dataset.tensors[0])
train_metric = f1_func(pred_y, torch.tensor(train_data_loader.dataset.tensors[1], dtype=torch.int32))
print("train_metric", train_metric)

train loss nan test f1 0.485
train loss nan test f1 0.485
train loss nan test f1 0.485
train loss nan test f1 0.485
train loss nan test f1 0.485
TEST RESULT:
	Result f1=0.485
	precision and recall=(0.471, 0.5)


C:\Users\Norma\AppData\Local\Temp\ipykernel_9380\3449417452.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_metric = f1_func(pred_y, torch.tensor(train_data_loader.dataset.tensors[1], dtype=torch.int32))


train_metric tensor(0.3291, device='cuda:0')


In [19]:
torch.cuda.empty_cache()

In [20]:
torch.max(pred_y[:, 1]), torch.min(pred_y[:, 1])

(tensor(nan, device='cuda:0', dtype=torch.float64, grad_fn=<MaxBackward1>),
 tensor(nan, device='cuda:0', dtype=torch.float64, grad_fn=<MinBackward1>))

In [21]:
torch.argmax(pred_y, dim=1).sum()

tensor(0, device='cuda:0')

In [22]:
## Making submission
test_table = test.drop(columns=["ST_YEAR", "SEMESTER", "TYPE_NAME"])
test_table = test_table.set_index("ISU")
t = students_table[students_table["ISU"].isin(test_table.index)]
t = t.drop(columns=["DISC_ID", "DEBT", "MARK", "MAIN_PLAN", "PRED_ID"])
t = t.drop_duplicates(subset="ISU")
t = t.set_index("ISU")
t = t.loc[test_table.index]
t = pd.concat([t, test_table], axis=1)
t = t.reset_index().drop(columns="ISU")
t = t.loc[:, train_x.columns]
t

,DISC_ID,ADMITTED_EXAM_1,ADMITTED_EXAM_2,ADMITTED_EXAM_3,9028324525774925341,7835846297793492094,14433121874407032269,1601392918367593206,9848012366215097300,8955667882044263414,...,TYPE_NAME_Дифференцированный зачет,TYPE_NAME_Зачет,TYPE_NAME_Курсовой проект,TYPE_NAME_Экзамен,ADMITTED_SUBJECT_PRIZE_LEVEL_0.0,ADMITTED_SUBJECT_PRIZE_LEVEL_1.0,ADMITTED_SUBJECT_PRIZE_LEVEL_2.0,ADMITTED_SUBJECT_PRIZE_LEVEL_3.0,ADMITTED_SUBJECT_PRIZE_LEVEL_4.0,ADMITTED_SUBJECT_PRIZE_LEVEL_ЕГЭ
0,552619236026332123,78.000000,83.000000,87.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,1
1,5702509582319458883,78.000000,83.000000,87.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,1
2,7735626259430674628,78.000000,83.000000,87.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,1
3,8955667882044263414,78.000000,83.000000,87.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,1
4,9075687484357010791,78.000000,83.000000,87.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30908,9075687484357010791,80.381229,82.389157,86.808098,0.0,4.0,0.0,4.0,4.0,0.0,...,0,0,0,1,0,0,1,0,0,0
30909,11240958499619480192,80.381229,82.389157,86.808098,0.0,4.0,0.0,4.0,4.0,0.0,...,0,0,0,1,0,0,1,0,0,0
30910,11511957009955804990,80.381229,82.389157,86.808098,0.0,4.0,0.0,4.0,4.0,0.0,...,0,0,0,1,0,0,1,0,0,0
30911,11807846188624377660,80.381229,82.389157,86.808098,0.0,4.0,0.0,4.0,4.0,0.0,...,0,0,0,1,0,0,1,0,0,0


In [23]:
t_tensor = torch.tensor(t.to_numpy(), device=device)
pred_test = model.predict(t_tensor).detach().cpu().numpy()
pred_test = np.argmax(pred_test, axis=1)
sample_submission["DEBT"] = pred_test
sample_submission.to_csv("./my_submission.csv", index=False)

In [24]:
sample_submission


,ID,DEBT
0,ISU:22160896830459167 | ST_YEAR:2021 | DISC_ID...,0
1,ISU:22160896830459167 | ST_YEAR:2021 | DISC_ID...,0
2,ISU:22160896830459167 | ST_YEAR:2021 | DISC_ID...,0
3,ISU:22160896830459167 | ST_YEAR:2021 | DISC_ID...,0
4,ISU:22160896830459167 | ST_YEAR:2021 | DISC_ID...,0
...,...,...
30908,ISU:18441846113897984349 | ST_YEAR:2021 | DISC...,0
30909,ISU:18441846113897984349 | ST_YEAR:2021 | DISC...,0
30910,ISU:18441846113897984349 | ST_YEAR:2021 | DISC...,0
30911,ISU:18441846113897984349 | ST_YEAR:2021 | DISC...,0


In [25]:
pred_test.sum()

0

In [26]:
pred_test.shape

(30913,)